**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
import numpy as np
import qgrid
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

In [ ]:
kafun = pd.read_pickle("kafun04.pkl")
kafun.head()

## qgrid が便利

In [ ]:
qgrid.show_grid(kafun)

---

## dfを感じる

In [ ]:
kafun.describe()

<br>
<br>
<br>

奈良から強いパワーを感じる

京都も不穏
<br>
<br>


---


## datetime を indexにしておくと便利な機能が多い

- df.loc["2018"]  
- df.loc["2018/04"]  
- df.loc["2013":"2015"]  

In [ ]:
kafun.loc["2013":"2015"]

In [ ]:
kafun.loc["2018/03/15 19:00:00":"2018/3/15 21"] # 雑なフォーマットでも気が利く

---

# groupby でいろいろな軸から集計してみる
<br>

## 注) : groupby は、pandasにおいてかなり大事な機能だが、
## 解説にはそれなりの時間がかかってしまうため、今回はgroupbyでできることの雰囲気を紹介

---

## 全データを 0-23時の軸で集計( median )

### lambda式を使って、dfの列に無い、オリジナルのgroupを作る

In [ ]:
kafun.groupby(lambda x:x.hour).median().plot(figsize=(20,9));

## 全データを年ごとにざっくり集計( median )

In [ ]:
kafun.groupby(lambda x:x.year).median().plot(figsize=(20,9));

## weekofyear（週番号 1-52）や weekday(曜日の数値),weekday_name(曜日名) とかも知ってると便利

In [ ]:
kafun.groupby(lambda x:x.weekday_name).sum().plot(figsize=(20,9), legend=True);

In [ ]:
kafun.groupby(lambda x:x.weekofyear).sum().plot(figsize=(20,9), legend=True);

---

# 年と月を含む独自のgroupを作る

2016年4月 → 201604 とか。

### year *100 + month にすれば単純にあらわせる

In [ ]:
kafun.groupby(lambda x:x.year*100+x.month).median().plot(figsize=(20,7));

### 数値のままだと、途中の多くな空欄になるので上記のようなグラフ
### stringにしてしまうことで、数値のような連続的じゃない、ただのラベルということにする。

In [ ]:
kafun.groupby(lambda x:str(x.year*100+x.month)).median().plot(figsize=(20,7));

---


# 複雑なgroupbyもちょっと紹介

## 各拠点の、毎年3月の 0-23時の分布(平均)を出す。

1. 毎年の3月を抜き出す
1. 0-23時の平均

<br>

### loc[ bool の配列 ] で絞る

In [ ]:
march = kafun.loc[kafun.index.month==3]
march


##  query も良い。 

####  \[　\] や　" "　の記述が少なくて簡潔に書けるので楽。
#### 複雑な絞りのパターンであるほど、評価が一回でまとめて行われるため処理としても少し速いとか（docに書いてあった気がするが、うろ覚え・・・）


<br>


In [ ]:
march = kafun.query("index.dt.month==3")  # いつもは不要だが、columns みたいに indexにもdtが必要
march

In [ ]:
march.groupby(lambda x:x.hour).median().plot(figsize=(20,7));

---

# さらに、年ごとにわけて表示する ( groupby が大好きな人用 )

* gruops
* get_group

を使いこなす

In [ ]:
group_year = march.groupby(lambda x:x.year)

.groups で groupby した時の key と 対応する indexが取れる

In [ ]:
group_year.groups

## group名と、get_gropupを使うと 該当のdfが取得できる！

### 小ネタ agg/aggregate なら集計関数をstringで指定できるので、 グラフのtitleと変数を統一できて、あとあとミスりにくい。

In [ ]:
agg_func_name = "mean"
for gr_name, year_kafun_index  in group_year.groups.items():
    #print(gr_name,year_kafun)
    year_kafun = group_year.get_group(gr_name)
    year_kafun.groupby(lambda x:x.hour).agg(agg_func_name).plot(
        figsize=(20,3), 
        title=str(gr_name)+"/03  hourly kafun ("+agg_func_name+")");
    plt.show(); # plot の確定を明示的にした。


---


# 毎 "時"  を   毎 "日"　に落とす down sampling ( hourly -> daily )
### datetime が index である必要がある。

type(df.index) が DatetimeIndex (pandas.core.indexes.datetimes.DatetimeIndex) ならOK 

In [ ]:
type(kafun.index)

In [ ]:
daily_resampler = kafun.resample("1D") # D, 24h でもOK 
daily_resampler

### どんな値を抽出するかの集計の式:  sum, mean, median, first, last, ...

In [ ]:
daily_resampler.first() # head(1) みたいなもの、ここでは、0時になる。 逆に last() なら 23時の値

# 平均で集計

In [ ]:
daily_resampler.mean().plot(figsize=(20,8));

<br>
<br>
<br>
<br>
<br>

# 結論：花粉辛い。
### [奈良(吉野保健所)](https://goo.gl/maps/ePz4zeezduR2) はすごい。 （意外に、衛星写真は山だらけでもない。）

<br>
<br>
<br>
<br>
